In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.9 MB/s eta 0:00:00


In [ ]:
SAVE_DIR = '/content/drive/MyDrive/BigData/Data/Model_Data'

In [ ]:
import pandas as pd
from torch_geometric.data import HeteroData
from collections import defaultdict
import torch

In [ ]:
kg_df = pd.read_csv("/content/drive/MyDrive/BigData/Data/Model_Data/kg_final.txt", sep='\t', header=None, names=['h', 'r', 't'])
kg_df.head()

,h,r,t
0,1518,0,2855
1,580,0,2854
2,1782,0,2837
3,2401,0,2847
4,636,0,2828


In [ ]:
interaction_df = pd.read_csv("/content/drive/MyDrive/BigData/Data/Model_Data/mapped_user_course.csv")
interaction_df.head()

,user,course,timestamp
0,27868,884,2020-02-06 10:31:52
1,70491,123,2020-03-02 23:41:20
2,33794,674,2019-11-14 19:19:44
3,33794,901,2020-07-17 10:22:11
4,22923,910,2020-03-27 18:47:47


In [ ]:
data = HeteroData()
edge_dict = defaultdict(list)

relation_mapping = {
    0: 'has_field',
    1: 'has_concept',
    2: 'taught_by',
    3: 'belongs_to_school',
    4: 'has_topic',
}

for _, row in kg_df.iterrows():
    course_id = int(row['h'])
    entity_id = int(row['t'])
    relation = int(row['r'])

    if relation == 0:
        entity_type = 'field'
    elif relation == 1:
        entity_type = 'concept'
    elif relation == 2:
        entity_type = 'teacher'
    elif relation == 3:
        entity_type = 'school'
    else:
        entity_type = 'topic'

    if entity_id not in entity_id_maps[entity_type]:
        entity_id_maps[entity_type][entity_id] = entity_id_counters[entity_type]
        entity_id_counters[entity_type] += 1

    normalized_entity_id = entity_id_maps[entity_type][entity_id]
    edge_type = ('course', relation_mapping[relation], entity_type)
    edge_dict[edge_type].append((course_id, entity_id))

for _, row in interaction_df.iterrows():
    user_id = int(row['user'])
    course_id = int(row['course'])

    edge_type = ('course', 'registered_by', 'user')
    edge_dict[edge_type].append((course_id, user_id))

for edge_type, edges in edge_dict.items():
    src, dst = zip(*edges)
    data[edge_type].edge_index = torch.tensor([src, dst], dtype=torch.long)

In [ ]:
data

HeteroData(
  (course, has_field, field)={ edge_index=[2, 472] },
  (course, has_concept, concept)={ edge_index=[2, 63303] },
  (course, taught_by, teacher)={ edge_index=[2, 273] },
  (course, belongs_to_school, school)={ edge_index=[2, 2319] },
  (course, has_topic, topic)={ edge_index=[2, 2807] },
  (course, registered_by, user)={ edge_index=[2, 1996390] }
)

In [ ]:
from torch_geometric.nn import MetaPath2Vec
from torch_geometric.transforms import ToUndirected

metapaths = [
    ('course', 'registered_by', 'user'),
    ('user', 'rev_registered_by', 'course'),
    ('course', 'has_concept', 'concept'),
    ('concept', 'rev_has_concept', 'course'),
    ('course', 'has_field', 'field'),
    ('field', 'rev_has_field', 'course'),
]

transform = ToUndirected()
data = transform(data)

model = MetaPath2Vec(data.edge_index_dict, embedding_dim=256, metapath=metapaths, walk_length=20,
                     context_size=5, walks_per_node=10, num_negative_samples=5,
                     sparse=True)

In [ ]:
from tqdm import tqdm
import os

loader = model.loader(batch_size=256, shuffle=True, num_workers=2)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

best_loss = float('inf')
patience = 5
wait = 0

def train():
    model.train()
    total_loss = 0
    with tqdm(total=len(loader), desc='Training') as pbar:
        for pos_rw, neg_rw in loader:
            optimizer.zero_grad()
            loss = model.loss(pos_rw, neg_rw)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pbar.update(1)
    return total_loss / len(loader)

for epoch in range(1, 51):
    loss = train()
    if loss < best_loss:
        best_loss = loss
        wait = 0
        torch.save(model.state_dict(), os.path.join(SAVE_DIR, 'best_metapath2vec.pt'))  # lưu lại checkpoint tốt nhất
    else:
        wait += 1
        if wait >= patience:
            print(f"Early stopping at epoch {epoch}")
            break
    print(f'Epoch: {epoch}, Loss: {loss:.4f}')

Training: 100%|██████████| 12/12 [01:35<00:00,  7.99s/it]


Epoch: 1, Loss: 9.2351


Training: 100%|██████████| 12/12 [01:25<00:00,  7.12s/it]


Epoch: 2, Loss: 8.5967


Training: 100%|██████████| 12/12 [01:26<00:00,  7.17s/it]


...

Epoch: 48, Loss: 1.0337


Training: 100%|██████████| 12/12 [01:33<00:00,  7.81s/it]


Epoch: 49, Loss: 1.0164


Training: 100%|██████████| 12/12 [01:27<00:00,  7.27s/it]


Epoch: 50, Loss: 0.9928


In [ ]:
model.eval()
course_emb = model('course')

embed_dim = course_emb.size(1)

padding_emb = torch.zeros(1, embed_dim)

mask_emb = torch.mean(course_emb, dim=0, keepdim=True)

full_course_emb = torch.cat([padding_emb, course_emb, mask_emb], dim=0)

torch.save(full_course_emb, os.path.join(SAVE_DIR, 'course_embeddings.pt'))